In [8]:
import sys
!{sys.executable} -m pip install pyvis

In [2]:
import pandas as pd
from pyvis.network import Network
import networkx as nx

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
stats_df = pd.read_csv('CombinedStats.csv', index_col=0)
stats_df.head()

sched_df = pd.read_csv('Conference_Games.csv', index_col=0)
sched_df.head()

,Rank,School,Games Played,Wins,Losses,W-L%,SRS,SOS,Conf W,Conf L,...,O_3PAr,O_TS%,O_TRB%,O_AST%,O_STL%,O_BLK%,O_eFG%,O_TOV%,O_ORB%,O_FT/FGA
0,1,Abilene Christian,25,10,15,0.400,-5.45,-1.07,5.0,9.0,...,0.314,0.554,53.3,47.5,9.2,8.4,0.518,18.7,30.3,0.291
1,2,Air Force,24,8,16,0.333,-3.78,-0.66,1.0,11.0,...,0.380,0.575,51.9,52.7,9.2,10.6,0.539,16.1,29.8,0.275
2,3,Akron,25,19,6,0.760,5.14,-1.95,11.0,1.0,...,0.370,0.507,46.2,49.0,9.2,7.6,0.479,16.2,25.1,0.189
3,4,Alabama,25,18,7,0.720,24.52,10.64,10.0,2.0,...,0.363,0.526,46.7,44.1,10.4,12.3,0.485,14.2,30.5,0.268
4,5,Alabama A&M,25,7,18,0.280,-15.79,-6.15,6.0,6.0,...,0.374,0.558,51.2,55.2,13.2,8.7,0.508,16.6,34.1,0.355


,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes
0,"Thu, Dec 28, 2023",Southern California,74.0,Oregon,82.0,NaN,NaN
1,"Thu, Dec 28, 2023",UCLA,69.0,Oregon State,62.0,NaN,NaN
2,"Fri, Dec 29, 2023",Arizona,100.0,California,81.0,NaN,NaN
3,"Fri, Dec 29, 2023",Washington,69.0,Colorado,73.0,NaN,NaN
4,"Fri, Dec 29, 2023",Arizona State,76.0,Stanford,73.0,NaN,NaN


In [4]:
sched_df['Score_Diff'] = sched_df['PTS'] - sched_df['PTS.1']

visitor = list(sched_df['Visitor/Neutral'])
home = list(sched_df['Home/Neutral'])
score_diff = list(sched_df['Score_Diff'])

winners = []
losers = []
tie = []

for i in range(len(score_diff)):
    
    if score_diff[i] > 0:
        winners.append(visitor[i])
        losers.append(home[i])
        tie.append(False)

    elif score_diff[i] < 0:
        winners.append(home[i])
        losers.append(visitor[i])
        tie.append(False)

    else:
        winners.append('None')
        losers.append('None')
        tie.append(True)

sched_df['Winner'] = winners
sched_df['Loser'] = losers
sched_df['Tie'] = tie
sched_df.head()

,Date,Visitor/Neutral,PTS,Home/Neutral,PTS.1,OT,Notes,Score_Diff,Winner,Loser,Tie
0,"Thu, Dec 28, 2023",Southern California,74.0,Oregon,82.0,NaN,NaN,-8.0,Oregon,Southern California,False
1,"Thu, Dec 28, 2023",UCLA,69.0,Oregon State,62.0,NaN,NaN,7.0,UCLA,Oregon State,False
2,"Fri, Dec 29, 2023",Arizona,100.0,California,81.0,NaN,NaN,19.0,Arizona,California,False
3,"Fri, Dec 29, 2023",Washington,69.0,Colorado,73.0,NaN,NaN,-4.0,Colorado,Washington,False
4,"Fri, Dec 29, 2023",Arizona State,76.0,Stanford,73.0,NaN,NaN,3.0,Arizona State,Stanford,False


In [5]:
# Initialize Network
net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", filter_menu=True)
net.force_atlas_2based()
net.toggle_physics(True)

# Node Data
all_schools = stats_df['School']
net.add_nodes(all_schools)
net.add_node('None')

# Edge Data 

# Source Node
source = sched_df['Visitor/Neutral']
# Target Node
target = sched_df['Home/Neutral']
# Weight Date
weights = abs(sched_df['Score_Diff'])

edge_data = zip(source, target, weights)
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    net.add_node(src, src, title=src)
    net.add_node(dst, dst, title=dst)
    net.add_edge(src, dst, value=w)


# add neighbor data to node hover data
neighbor_map = net.get_adj_list()

for node in net.nodes:
    node["title"] = " Neighbors:\n" + "\n".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

In [6]:
net.show('ScheduleGraphViz.html', notebook=False)

ScheduleGraphViz.html
